In [1]:
%load_ext autoreload
%load_ext dotenv
%dotenv
!huggingface-cli login --token $HUGGING_FACE_TOKEN

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/izlobin/.cache/huggingface/token
Login successful


In [2]:
# %%capture
# %pip install transformers evaluate
# %pip install nltk absl-py rouge_score
# %pip install bleu sacrebleu
# %pip install sacremoses
# %pip install scipy
# %pip install sentencepiece
# %pip install optimum auto-gptq
# %pip install scikit-learn
# %pip install einops
# %pip install bitsandbytes
# %pip install accelerate
# %pip install pynvml

In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
from pprint import pprint

import evaluate
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from huggingface_hub import HfApi
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    BartForCausalLM,
    BartModel,
    BartTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    T5ForConditionalGeneration,
    T5Tokenizer,
    T5TokenizerFast,
)

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

from utils.dataset import get_iterater_samples_simplified, get_iterater_samples_with_instruction
from utils.metric import calculate_scores
from utils.monitoring import calculate_utilization, format_utilization_narrow, print_utilization

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pprint(f"Device: {device}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
'Device: cuda'


## Loading models

### Loading BART
* https://huggingface.co/facebook/bart-large

In [4]:
from transformers import BartTokenizer, AutoModelForSeq2SeqLM

model_name = "t5-large"
model_repo = f"google-t5"
model_id = f"{model_repo}/{model_name}"
model_checkpoint = f"{model_repo}/{model_name}"
model_path = f"{model_repo}_{model_name}"
print(f"model_name: {model_name}," f"model_id: {model_id}," f"model_path: {model_path}")

tokenizer = T5TokenizerFast.from_pretrained(model_id)
# tokenizer.add_eos_token = True
# tokenizer.pad_token_id = tokenizer.eos_token_id
# tokenizer.padding_side = "right"
print(type(tokenizer))

# model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map=0, max_memory={0: "20GIB"})
model = T5ForConditionalGeneration.from_pretrained(model_id, device_map=0)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")
print(type(model))
print(model.config)

model_name: t5-large,model_id: google-t5/t5-large,model_path: google-t5_t5-large
<class 'transformers.models.t5.tokenization_t5_fast.T5TokenizerFast'>


/home/izlobin/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:171: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


<class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>
T5Config {
  "_name_or_path": "google-t5/t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 4096,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    

## Loading dataset

### Grammarly/coedit dataset

In [5]:
from datasets import DatasetDict, load_dataset

# full_dataset = load_dataset("grammarly/coedit")
# print(full_dataset)

# train_dataset = load_dataset("grammarly/coedit", split="train[:50000]")
# test_dataset = load_dataset("grammarly/coedit", split="train[10000:]")
# # test_dataset = load_dataset("grammarly/coedit", split="validation")

all_dataset = load_dataset("grammarly/coedit", split="train+validation")
# print(all_dataset)

# print()
# print(f"train set {set(all_dataset['task'])}")
# print(f"total len: {len(all_dataset)}")
# print(f"gec len: {len(all_dataset.filter(lambda x: x['task'] == 'gec'))}")
# print(f"simplification len: {len(all_dataset.filter(lambda x: x['task'] == 'simplification'))}")
# print(f"clarity len: {len(all_dataset.filter(lambda x: x['task'] == 'clarity'))}")
# print(f"coherence len: {len(all_dataset.filter(lambda x: x['task'] == 'coherence'))}")
# print(f"paraphrase len: {len(all_dataset.filter(lambda x: x['task'] == 'paraphrase'))}")
# print(f"neutralize len: {len(all_dataset.filter(lambda x: x['task'] == 'neutralize'))}")
# print()

# train_ratio = 0.001
# test_ratio = 0.0001
train_ratio = 0.1
test_ratio = 0.01
# train_ratio = 0.9
# test_ratio = 0.1

gec_dataset = all_dataset.filter(lambda x: x["task"] == "gec")
train_gec_dataset = gec_dataset.select(range(0, int(train_ratio * len(gec_dataset))))
test_gec_dataset = gec_dataset.select(range(int((1 - test_ratio) * len(gec_dataset)), len(gec_dataset)))

simplification_dataset = all_dataset.filter(lambda x: x["task"] == "simplification")
train_simplification_dataset = simplification_dataset.select(range(0, int(train_ratio * len(simplification_dataset))))
test_simplification_dataset = simplification_dataset.select(
    range(int((1 - test_ratio) * len(simplification_dataset)), len(simplification_dataset))
)

clarity_dataset = all_dataset.filter(lambda x: x["task"] == "clarity")
train_clarity_dataset = clarity_dataset.select(range(0, int(train_ratio * len(clarity_dataset))))
test_clarity_dataset = clarity_dataset.select(range(int((1 - test_ratio) * len(clarity_dataset)), len(clarity_dataset)))

coherence_dataset = all_dataset.filter(lambda x: x["task"] == "coherence")
train_coherence_dataset = coherence_dataset.select(range(0, int(train_ratio * len(coherence_dataset))))
test_coherence_dataset = coherence_dataset.select(
    range(int((1 - test_ratio) * len(coherence_dataset)), len(coherence_dataset))
)

paraphrase_dataset = all_dataset.filter(lambda x: x["task"] == "paraphrase")
train_paraphrase_dataset = paraphrase_dataset.select(range(0, int(train_ratio * len(paraphrase_dataset))))
test_paraphrase_dataset = paraphrase_dataset.select(
    range(int((1 - test_ratio) * len(paraphrase_dataset)), len(paraphrase_dataset))
)

neutralize_dataset = all_dataset.filter(lambda x: x["task"] == "neutralize")
neutralize_dataset_split = int(train_ratio * len(neutralize_dataset))
train_neutralize_dataset = neutralize_dataset.select(range(0, int(train_ratio * len(neutralize_dataset))))
test_neutralize_dataset = neutralize_dataset.select(
    range(int((1 - test_ratio) * len(neutralize_dataset)), len(neutralize_dataset))
)

from datasets import concatenate_datasets

train_dataset = concatenate_datasets(
    [
        train_gec_dataset,
        train_simplification_dataset,
        train_clarity_dataset,
        train_coherence_dataset,
        train_paraphrase_dataset,
        train_neutralize_dataset,
    ]
)
train_dataset = train_dataset.map(
    lambda item: {
        "input": item["src"],
        "reference": item["tgt"],
        "references": [item["tgt"]],
    },
    remove_columns=["src", "tgt", "_id"],
)
print(f"train set {set(train_dataset['task'])}")
print(train_dataset)

test_dataset = concatenate_datasets(
    [
        test_gec_dataset,
        test_simplification_dataset,
        test_clarity_dataset,
        test_coherence_dataset,
        test_paraphrase_dataset,
        test_neutralize_dataset,
    ]
)
test_dataset = test_dataset.map(
    lambda item: {
        "input": item["src"],
        "reference": item["tgt"],
        "references": [item["tgt"]],
    },
    remove_columns=["src", "tgt", "_id"],
)
print(f"test set {set(test_dataset['task'])}")
print(test_dataset)

dataset = DatasetDict({"train": train_dataset, "test": test_dataset})
# dataset = dataset.rename_column("task", "label")
# dataset = dataset.map(
#     lambda item: {
#         "input": item["src"],
#         "reference": item["tgt"],
#         "references": [item["tgt"]],
#     },
#     remove_columns=["src", "tgt", "_id"],
# )
print(dataset)
print(dataset["train"][0])

train set {'gec', 'coherence', 'simplification', 'neutralize', 'clarity', 'paraphrase'}
Dataset({
    features: ['task', 'input', 'reference', 'references'],
    num_rows: 7076
})
test set {'gec', 'coherence', 'simplification', 'neutralize', 'clarity', 'paraphrase'}
Dataset({
    features: ['task', 'input', 'reference', 'references'],
    num_rows: 711
})
DatasetDict({
    train: Dataset({
        features: ['task', 'input', 'reference', 'references'],
        num_rows: 7076
    })
    test: Dataset({
        features: ['task', 'input', 'reference', 'references'],
        num_rows: 711
    })
})
{'task': 'gec', 'input': 'Remove all grammatical errors from this text: For example, countries with a lot of deserts can terraform their desert to increase their habitable land and using irrigation to provide clean water to the desert.', 'reference': 'For example, countries with a lot of deserts can transform their desert to increase their habitable land and use irrigation to provide clean wate

In [6]:
# find the longest sequence in the dataset
max_input_length = max(len(tokenizer.encode(item["input"])) for item in dataset["train"])
print(f"max_input_length train: {max_input_length}")
max_input_length = max(len(tokenizer.encode(item["input"])) for item in dataset["test"])
print(f"max_input_length test: {max_input_length}")

max_input_length train: 154
max_input_length test: 278


In [7]:
from datasets import Dataset

train_lists_map = {}

for task in set(train_dataset['task']):
    train_lists_map[task] = []

for item in dataset["train"]:
    train_lists_map[item["task"]].append(item)

train_dataset_map = {}
for task, list in train_lists_map.items():
    train_dataset_map[task] = Dataset.from_list(list)
# print(train_dataset_map)

train_dataset_dict = DatasetDict(train_dataset_map)
# print(train_dataset_dict)

# for task, ds in train_dataset_dict.items():
#     print(f"{task}: {ds}")

for task in set(train_dataset['task']):
    print(f"train/{task}: {len(train_lists_map[task])}")

train/gec: 2030
train/coherence: 1061
train/simplification: 1144
train/neutralize: 1127
train/clarity: 125
train/paraphrase: 1589


In [8]:
from datasets import Dataset

test_lists_map = {}

for task in set(train_dataset['task']):
    test_lists_map[task] = []

for item in dataset["test"]:
    test_lists_map[item["task"]].append(item)

test_dataset_map = {}
for task, list in test_lists_map.items():
    test_dataset_map[task] = Dataset.from_list(list)
# print(test_dataset_map)

test_dataset_dict = DatasetDict(test_dataset_map)
# print(test_dataset_dict)

# for task, ds in test_dataset_dict.items():
#     print(f"{task}: {ds}")

for task in set(train_dataset['task']):
    print(f"test/{task}: {len(test_lists_map[task])}")

test/gec: 204
test/coherence: 107
test/simplification: 115
test/neutralize: 113
test/clarity: 13
test/paraphrase: 159


In [9]:
import numpy as np
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling, DataCollatorForSeq2Seq

print(f"pad_token_id: {tokenizer.pad_token_id}")
print(f"padding_side: {tokenizer.padding_side}")


def process_dataset(batch):
    model_inputs = tokenizer(batch["input"], max_length=1024, padding=True)
    labels = tokenizer(text_target=batch["reference"], max_length=1024, padding=True).input_ids
    # model_inputs = tokenizer(batch["input"], padding=True)
    # labels = tokenizer(text_target=batch["reference"], padding=True).input_ids
    # model_inputs = tokenizer(batch["input"])
    # labels = tokenizer(text_target=batch["reference"]).input_ids
    model_inputs["labels"] = labels
    return model_inputs


# processed_dataset = data.map(preprocess_function, batched=True, batch_size=10, remove_columns=data["train"].column_names)
processed_dataset = dataset.map(process_dataset, batched=True, batch_size=10, remove_columns=train_dataset.column_names)
# processed_dataset = dataset.map(process_dataset, batched=True, batch_size=10)
print(processed_dataset)
print(len(processed_dataset["train"]["input_ids"][0]), processed_dataset["train"]["input_ids"][0])
print(len(processed_dataset["train"]["attention_mask"][0]), processed_dataset["train"]["attention_mask"][0])
print(len(processed_dataset["train"]["labels"][0]), processed_dataset["train"]["labels"][0])

data_collator = DataCollatorForSeq2Seq(tokenizer, return_tensors="pt", model=model)
# data_collator = DataCollatorForSeq2Seq(tokenizer, return_tensors="pt", padding=True)

# dataloader = DataLoader(processed_dataset, batch_size=5, collate_fn=data_collator)
# for batch in dataloader:
#     print(batch)
#     break

pad_token_id: 0
padding_side: right


Map:   0%|          | 0/711 [00:00<?, ? examples/s]

/home/izlobin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2706: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7076
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 711
    })
})
55 [10002, 66, 3, 5096, 4992, 138, 6854, 45, 48, 1499, 10, 242, 677, 6, 1440, 28, 3, 9, 418, 13, 9980, 7, 54, 3, 12829, 2032, 70, 9980, 12, 993, 70, 7386, 179, 1322, 11, 338, 21455, 12, 370, 1349, 387, 12, 8, 9980, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
55 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
44 [242, 677, 6, 1440, 28, 3, 9, 418, 13, 9980, 7, 54, 3343, 70, 9980, 12, 993, 70, 7386, 179, 1322, 11, 169, 21455, 12, 370, 1349, 387, 12, 8, 9980, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Pre-test inference

In [10]:
max_batch = 2
max_length = 350

for task, batch in test_dataset_dict.items():
    print()
    print(f">> {task}")
    batch_size = len(batch) if len(batch) < max_batch else max_batch
    input_batch = batch.select(range(batch_size))
    print(f"input: {input_batch['input']}")

    input = tokenizer(input_batch["input"], padding=True, return_tensors="pt").to(device)
    outputs = model.generate(input.input_ids, max_length=max_length)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(f"result: {result}")

    break


>> gec
input: ["Fix grammaticality: How many times do we turn before the television and we watch different commercial ad beyond different enterprises that are permanently appealing the youngster to have a new toy or a new doll and put a pressure on the parent's to they will have, most over the time, to spending money on something that will bring to the child a momentaneous happiness.", 'Fix grammaticality of the sentence: I thinking you should go on holidays for your friend before you can has more fun without upon your parents In fact with your friends you can did what you are keen on: swimming, going shopping and going to the disco, instead of having a boring holiday with your parent, who want you to did the same thing they do I hope you will choose what is the best by you']
result: [':: and we see different commercial grammaticality: grammaticality: grammaticality:: we turn before the television and we watch different commercial beyond different enterprises that are permanently appe

## Training

In [11]:
%reload_ext autoreload

training_model_repo = f"iliazlobin"
training_model_name: str = f"{model_name}-coedit"
training_model_id = f"{training_model_repo}/{training_model_name}"
training_model_checkpoint = f"{training_model_id}"
training_model_path = f"{training_model_repo}_{training_model_name}"
print(
    f"training_model_name: {training_model_name}, "
    f"training_model_id: {training_model_id}, "
    f"training_model_path: {training_model_path}"
)

utilization = calculate_utilization()
utilization_str = format_utilization_narrow(utilization)
print(
    f"total/used/cuda/res/ram (Gb): {utilization_str['total_memory']}/{utilization_str['memory_used']}/"
    f"{utilization_str['cuda_allocated']}/{utilization_str['cuda_reserved']}/{utilization_str['ram_usage']}"
)

available_memory = utilization["total_memory"] - utilization["memory_used"]
recommended_fraction = available_memory / utilization["total_memory"]

actual_fraction = 0.95
torch.cuda.set_per_process_memory_fraction(actual_fraction, 0)

print(
    f"total/used/available memory (Gb): {utilization['total_memory']/1024**3:.2f}/"
    f"{utilization['memory_used']/1024**3:.2f}/{available_memory/1024**3:.2f}"
)
print(f"recommended/actual fraction: {recommended_fraction:.2f}/{actual_fraction:.2f}")

training_model_name: t5-large-coedit, training_model_id: iliazlobin/t5-large-coedit, training_model_path: iliazlobin_t5-large-coedit
total/used/cuda/res/ram (Gb): 10.00/4.46/2.76/2.91/15.13
total/used/available memory (Gb): 10.00/4.46/5.54
recommended/actual fraction: 0.55/0.95


In [1]:
training_model = model
# print(type(training_model))
# print(training_model.config)

total_params = sum(p.numel() for p in training_model.parameters())
total_trainable_params = sum(p.numel() for p in training_model.parameters() if p.requires_grad)
# print(f"Total/trainable params: {total_params}/{total_trainable_params}")


rouge_metric = evaluate.load("rouge")
# glue_metric = evaluate.load("glue", "stsb")
sacreblue_metric = evaluate.load("sacrebleu")
sari_metric = evaluate.load("sari")
em_metric = evaluate.load("exact_match")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    print(decoded_preds)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    print(decoded_labels)

    rouge_score = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    sacreblue_score = sacreblue_metric.compute(predictions=decoded_preds, references=decoded_labels)
    # sari_score = sari_metric.compute(
    #     sources=processed_samples["input"],
    #     predictions=processed_samples["processed"],
    #     references=processed_samples["references"],
    # )
    em_score = em_metric.compute(predictions=decoded_preds, references=decoded_labels)

    report = {
        "rouge1": rouge_score["rouge1"],
        "rouge2": rouge_score["rouge2"],
        "rougeL": rouge_score["rougeL"],
        "rougeLsum": rouge_score["rougeLsum"],
        "sacreblue": sacreblue_score["score"],
        "memory_used": utilization["memory_used"] / 1024**2,
        "cuda_allocated": utilization["cuda_allocated"] / 1024**2,
        "cuda_reserved": utilization["cuda_reserved"] / 1024**2,
        "ram_usage": utilization["ram_usage"] / 1024**2,
        # "sari.sari": 0,
        "em": em_score["exact_match"],
    }

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    report["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in report.items()}

# debug
# trainer = Seq2SeqTrainer(
#     model=training_model,
#     train_dataset=processed_dataset["train"],
#     eval_dataset=processed_dataset["test"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
#     args=args,
# )
# predictions = trainer.predict(processed_dataset["train"].select(range(2)))
# print(type(predictions))
# print(predictions)
# metrics = compute_metrics((predictions.predictions, predictions.label_ids))
# print(metrics)

NameError: name 'model' is not defined

In [13]:
%%capture
%load_ext tensorboard
%tensorboard --logdir "model-{training_model_path}-train/runs"

In [2]:
train_path = f"model-{training_model_path}-train"
print(f"train_path: {train_path}")

train_size = len(processed_dataset["train"])
batch_size = 10
gradient_accumulation_steps = 4
eval_batch_size = 10
eval_accumulation_steps = 4

per_epoch_steps = train_size / (batch_size * gradient_accumulation_steps)

max_steps = 108
epochs = max_steps / per_epoch_steps
print(f"train_size: {train_size}, batch_size: {batch_size}, per_epoch_steps: {per_epoch_steps}, max_steps: {max_steps}, epochs: {epochs}")

# if epochs < 1:
#     raise Exception(f"Training doesn't cover the entire training dataset with {train_size} samples")

epochs = 1
epoch_total_steps = epochs * per_epoch_steps
print(f"train_size: {train_size}, batch_size: {batch_size}, per_epoch_steps: {per_epoch_steps}, epochs: {epochs}, epoch_total_steps: {epoch_total_steps}")


NameError: name 'training_model_path' is not defined

In [15]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

args = Seq2SeqTrainingArguments(
    output_dir=train_path,
    learning_rate=2e-5,
    weight_decay=0.01,
    # warmup_ratio=0.05,
    # optim="paged_adamw_8bit",
    fp16=True,
    gradient_checkpointing=True,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=eval_batch_size,
    eval_accumulation_steps=eval_accumulation_steps,
    num_train_epochs=epochs,
    # max_steps=max_steps,
    warmup_steps=1,
    logging_steps=10,
    # save_strategy="steps",
    # save_steps=5,
    evaluation_strategy="steps",
    eval_steps=50,
    report_to="tensorboard",
    push_to_hub_model_id=training_model_name,
    push_to_hub_organization="iliazlobin",
    push_to_hub=False,
    gradient_checkpointing_kwargs={
        "use_reentrant": False
    },  # Should be false for Lora (https://github.com/kohya-ss/sd-scripts/issues/323#issuecomment-1485073421)
)


trainer = Seq2SeqTrainer(
    model=training_model,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    args=args,
)
print(type(trainer))

model.config.use_cache = False
# trainer.train(resume_from_checkpoint=True)
trainer.train()

<class 'transformers.trainer_seq2seq.Seq2SeqTrainer'>


/home/izlobin/.local/lib/python3.10/site-packages/transformers/training_args.py:1859: FutureWarning: `--push_to_hub_model_id` and `--push_to_hub_organization` are deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case iliazlobin/t5-large-coedit).
  warnings.warn(
/home/izlobin/.local/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
5,0.000000
10,0.000000
15,0.000000
20,0.000000
25,0.000000
30,0.000000
35,0.000000
40,0.000000
45,0.000000
50,0.000000


In [66]:
%load_ext dotenv
%dotenv ../.env

!huggingface-cli login --token os.getenv(key="HUGGING_FACE_TOKEN")

print(type(trainer))
print(type(trainer.model))

# print(f"saving locally: {model-{training_model_path}}")
# trainer.save_model(f"model-{training_model_path}")

# print(f"pushing to hub: {training_model_id}")
# # trainer.save_model(f"model-{training_model_path}")
# trainer.push_to_hub(commit_message="test", model_name=training_model_name)
# trainer.push_to_hub(commit_message="test")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `huggingface-cli login --token os.getenv(key="HUGGING_FACE_TOKEN")'
BartConfig {
  "_name_or_path": "facebook/bart-large",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label":

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-66209bc6-1646073a1451ab1e624e9857;06828fd2-e37b-43b6-abbb-7e1b22580527)

Repository Not Found for url: https://huggingface.co/api/models/model-iliazlobin_bart-large-coedit-train.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.

## Test inference

In [17]:
model = trainer.model

max_batch = 2
max_length = 350

for task, batch in test_dataset_dict.items():
    print()
    print(f">> {task}")
    batch_size = len(batch) if len(batch) < max_batch else max_batch
    input_batch = batch.select(range(batch_size))
    print(f"input: {input_batch['input']}")

    input = tokenizer(input_batch["input"], padding=True, return_tensors="pt").to(device)
    outputs = model.generate(input.input_ids, max_length=max_length)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(f"result: {result}")

    break

/home/izlobin/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/izlobin/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...



>> paraphrase
input: ['Rewrite this sentence: Researchers and national competent authorities have developed a wide range of collections and related databases on genetic resources, breeds and plant varieties.', 'Write a paraphrased version of the sentence: The Commission report will be accompanied, if appropriate, by a legislative proposal.']
result: [': and breeds. and databases on genetic resources and breeds. and.::::. this sentence: Researchers and national competent authorities have developed a wide range of collections and related databases on genetic resources, breeds and plant varieties. on genetic resources,,::: and related databases on genetic resources, breeds and plant varieties. a wide range of on, the, and', 'report. The Commission report will be accompanied, if appropriate, by a legislative proposal. The Commission report will be accompanied, if appropriate.']


## Re-upload model

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BartForConditionalGeneration, BitsAndBytesConfig

model_name = "model-iliazlobin_bart-large-coedit-train"
model_repo = f"iliazlobin"
model_id = f"{model_repo}/{model_name}"
model_checkpoint = f"{model_repo}/{model_name}"
model_path = f"{model_repo}_{model_name}"
print(f"model_name: {model_name}," f"model_id: {model_id}," f"model_path: {model_path}")


tokenizer = BartTokenizer.from_pretrained(model_id)
# tokenizer.add_eos_token = True
# tokenizer.pad_token_id = tokenizer.eos_token_id
# tokenizer.padding_side = "left"
print(type(tokenizer))

tokenizer.push_to_hub(repo_id="iliazlobin/bart-large-coedit")

model = BartForConditionalGeneration.from_pretrained(model_id, device_map=0)
print(type(model))
print(model.config)
# print(model)

# !huggingface-cli login --token os.getenv(key="HUGGING_FACE_TOKEN")

# print(training_model_path)
# print(type(trainer))
# print(type(trainer.model))

model.push_to_hub(commit_message="train complete on coedit dataset", repo_id="iliazlobin/bart-large-coedit")
model.generation_config.push_to_hub(repo_id="iliazlobin/bart-large-coedit")

model_name: model-iliazlobin_bart-large-coedit-train,model_id: iliazlobin/model-iliazlobin_bart-large-coedit-train,model_path: iliazlobin_model-iliazlobin_bart-large-coedit-train
<class 'transformers.models.bart.tokenization_bart.BartTokenizer'>
<class 'transformers.models.bart.modeling_bart.BartForConditionalGeneration'>
BartConfig {
  "_name_or_path": "iliazlobin/model-iliazlobin_bart-large-coedit-train",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "enco

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


CommitInfo(commit_url='https://huggingface.co/iliazlobin/bart-large-coedit/commit/c2e558803e815433a4dcae0a9f84a3332c0804f7', commit_message='Upload config', commit_description='', oid='c2e558803e815433a4dcae0a9f84a3332c0804f7', pr_url=None, pr_revision=None, pr_num=None)